In [1]:
import pandas as pd

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import io

# Initialize an empty dataframe to store the results
df_list = []

# Loop through the entire year in 4-day increments
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 1, 2)

while start_date <= end_date:
    # Calculate the end date for the current 4-day period
    current_end_date = start_date + timedelta(days=3)
    if current_end_date > end_date:
        current_end_date = end_date
    
    # Format the dates as strings
    start_date_str = start_date.strftime('%Y-%m-%d')
    current_end_date_str = current_end_date.strftime('%Y-%m-%d')
    print(start_date)
    
    # Update the URL with the current date range
    url = f'https://data.elexon.co.uk/bmrs/api/v1/datasets/AGPT?publishDateTimeFrom={start_date_str}&publishDateTimeTo={current_end_date_str}&format=csv'
    
    headers = {
        'accept': 'text/plain',
    }
    # Make the API request
    response = requests.get(url, headers=headers)
    
    # Read the response content into a dataframe
    df = pd.read_csv(io.StringIO(response.text))
    
    # Append the dataframe to the list
    df_list.append(df)
    
    # Move to the next 4-day period
    start_date = current_end_date + timedelta(days=1)

# Concatenate all the dataframes into a single dataframe
final_df = pd.concat(df_list, ignore_index=True)

# Print the final dataframe
print(final_df)

2022-01-01 00:00:00
2022-01-05 00:00:00
2022-01-09 00:00:00
2022-01-13 00:00:00
2022-01-17 00:00:00
2022-01-21 00:00:00
2022-01-25 00:00:00
2022-01-29 00:00:00
2022-02-02 00:00:00
2022-02-06 00:00:00
2022-02-10 00:00:00
2022-02-14 00:00:00
2022-02-18 00:00:00
2022-02-22 00:00:00
2022-02-26 00:00:00
2022-03-02 00:00:00
2022-03-06 00:00:00
2022-03-10 00:00:00
2022-03-14 00:00:00
2022-03-18 00:00:00
2022-03-22 00:00:00
2022-03-26 00:00:00
2022-03-30 00:00:00
2022-04-03 00:00:00
2022-04-07 00:00:00
2022-04-11 00:00:00
2022-04-15 00:00:00
2022-04-19 00:00:00
2022-04-23 00:00:00
2022-04-27 00:00:00
2022-05-01 00:00:00
2022-05-05 00:00:00
2022-05-09 00:00:00
2022-05-13 00:00:00
2022-05-17 00:00:00
2022-05-21 00:00:00
2022-05-25 00:00:00
2022-05-29 00:00:00
2022-06-02 00:00:00
2022-06-06 00:00:00
2022-06-10 00:00:00
2022-06-14 00:00:00
2022-06-18 00:00:00
2022-06-22 00:00:00
2022-06-26 00:00:00
2022-06-30 00:00:00
2022-07-04 00:00:00
2022-07-08 00:00:00
2022-07-12 00:00:00
2022-07-16 00:00:00


In [3]:
final_df['SettlementDate'] = pd.to_datetime(final_df['SettlementDate'])
final_df['StartTime'] = pd.to_datetime(final_df['StartTime'])

In [4]:
final_df.sort_values(by=['StartTime'],ascending=True, inplace=True)

In [5]:
final_df=final_df.drop(columns=["Dataset","DocumentId","DocumentRevisionNumber","PublishTime","BusinessType","SettlementDate","SettlementPeriod"])

In [10]:
final_df["StartTime"].unique()

<DatetimeArray>
['2021-12-31 23:00:00+00:00', '2021-12-31 23:30:00+00:00',
 '2022-01-01 00:00:00+00:00', '2022-01-01 00:30:00+00:00',
 '2022-01-01 01:00:00+00:00', '2022-01-01 01:30:00+00:00',
 '2022-01-01 02:00:00+00:00', '2022-01-01 02:30:00+00:00',
 '2022-01-01 03:00:00+00:00', '2022-01-01 03:30:00+00:00',
 ...
 '2023-01-01 18:00:00+00:00', '2023-01-01 18:30:00+00:00',
 '2023-01-01 19:00:00+00:00', '2023-01-01 19:30:00+00:00',
 '2023-01-01 20:00:00+00:00', '2023-01-01 20:30:00+00:00',
 '2023-01-01 21:00:00+00:00', '2023-01-01 21:30:00+00:00',
 '2023-01-01 22:00:00+00:00', '2023-01-01 22:30:00+00:00']
Length: 11369, dtype: datetime64[ns, UTC]

In [16]:
# Pivot the DataFrame
pivot_df = final_df.pivot_table(index='StartTime', columns='PsrType', values='Quantity',aggfunc='first')

In [18]:
pivot_df=pivot_df.resample('h').mean()

In [21]:
pivot_df.isna().sum()

PsrType
Biomass                            3082
Fossil Gas                         3082
Fossil Hard coal                   3082
Fossil Oil                         3082
Hydro Pumped Storage               3082
Hydro Run-of-river and poundage    3082
Nuclear                            3082
Other                              3082
Solar                              3082
Wind Offshore                      3082
Wind Onshore                       3082
dtype: int64